# 2020-1 모두를 위한 인공지능 활용

## GEK-10109


### 프로젝트 7조
### 21500615 장관우 21900410 신호철 22000131 김예나






# 예비 보고서 
1. 프로젝트 선정계기

    a. 중국 우한에서부터 시작된 코로나 바이러스는 접촉 또는 비말에 의해서 감염된다.
    하지만 마스크를 착용하면 재채기나 기침으로 인한 바이러스 노출 위험을 낮추어 코로나 바이러스를
    조금이나마 예방할 수 있다
    
    b. 인공지능 기술을 활용하여 접촉하지 않고도 마스크 착용 유무를 판단 하여 코로나 예방에 도움이 되고자한다.
    
    
2. 프로젝트 개요


    a. 활용하는 데이터는 Kaggle에 있는 Data set으로, 마스크를 착용한 사람과 마스크를 미 착용한 사람의 이미지에 대한 데이터를 활용한다.
    
    b. CNN(Convolutional  Neural Network)을 이용한 이미지 학습 모델 생성, Classification(착용/ 미착용)을 활용한다.
    
    c. dlib와 openCV를 활용하여, 컴퓨터 비전응 이용한 영상처리와 얼굴인식을 하고, 학습 모델을 통해서 마스크 착용자와 마스크 비 착용자를 구별한다.

3. 기대효과

    a. 마스크를 착용하지 않은 사람들은 관리자에게 알려 건물의 출입을 금할 수 있게 한다.
    
    b. 집단 감염에 취약한 시설들(대형병원, 종교시설, 유흥업소 등등)에 마스크를 착용하지 않은 사람들에 대한 입장 제한을 통해
    집단 감염을 예방할 수 있다.
    
    c. 사람들에게 마스크 착용에 대한 경각심을 불러 일으킬 수 있다.

# Classfication 학습하기

다음 단계로 학습을 진행한다.

1. 데이터 전처리 + 정규화(Normalization)
2. CNN 모델 정의
3. 손실함수(Loss Function)와 Optimizer 정의
4. 학습 (Train)
5. 평가 (Validate)





# 프로젝트 구성

![Diagram](./Diagram.png)

1. CNN과 마스크 데이터셋을 이용하여 Classification 모델을 생성
2. Open CV의 얼굴인식 라이브러리를 이용하여, 얼굴을 인식
3. 얼굴 이미지를 추출하여, 얼굴 이미지를 PreProcessing과정을 통해, 이미지 가공 후, 모델에 투입
4. 0 - Mask 1- Non-Mask 라벨을 통해, Classfication 결과를 화면에 출력



# 모듈 버전 확인

In [10]:
import torch #1.5.0
import torchvision #0.6.0
import keras #2.3.1
import cv2 #4.2.0
import numpy as np #1.17.2

print(torch.__version__) 
print(torchvision.__version__)
print(keras.__version__)
print(cv2.__version__)
print(np.__version__)

1.5.0
0.6.0
2.3.1
4.2.0
1.17.2


# Dataset
현재 사용하는 데이터 셋 https://www.kaggle.com/ahmetfurkandemr/mask-datasets-v1/data




Data _ Mask
![Mask1](./Mask_Datasets/Train/Mask/1.PNG)
![Mask2](./Mask_Datasets/Train/Mask/15.PNG)
Data _ No MASK 
![Mask3](./Mask_Datasets/Train/No_Mask/1.PNG)
![Mask4](./Mask_Datasets/Train/No_Mask/250.PNG)
데이터 셋 구성

Training Data Set
Mask Data 350개
Non-Mask Data 400개

Validate Data Set
Mask Data 150개
Non-Mask Data 200개


# 데이터 전처리 및 정규화
현재 있는 이미지 데이터를 보다시피, 크기가 모두 다르기 때문에, 데이터를 사이즈를 일정한 크기로 Resize시켜야 한다. 그리고 Pytorch에서 학습을 진행시키기 위해서 numpy array형태에서 Torch Tensor로 변환, 학습을 위한 정규화 또한 필요하다. 
torchvision 모듈의 transforms class의 Compose 메소드를 이용하여, 이러한 데이터 전처리 작업을 진행을 위한 transform형태를 정한다.

In [1]:
from torchvision import transforms
import torch
import torchvision


trans = transforms.Compose([transforms.Resize((224,224)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.485,0.456,0.406), (0.229, 0.224, 0.225))
                           ])


위에서 정한 Transform형태에 따라 데이터를 torchvision의 Dataset형태로 데이터를 변환시킨다. 이때, Train/Validation 폴더에 있는 Mask와 No_Mask가 분류가 된다. (Label1 = 0 Mask), (Label2 = 1 No_mask)
Train_set 변수에 DataLoader형태로 데이터를 변수를 정한다. 이때, batch_size와 shuffle num_workers에 대한 정보를 넘긴다.

In [2]:
from torch.utils.data import DataLoader, Dataset
train_data =  torchvision.datasets.ImageFolder(root='./Mask_Datasets/train/', transform = trans)
test_data = torchvision.datasets.ImageFolder(root='./Mask_Datasets/Validation', transform = trans)
train_set = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))



## Neural Net 만들기

신경망에 1채널 이미지만 처리할수 있는 신경망을 3채널로 처리 -> 6채널로 처리 하도록 변경,  
44944 ->120 -> 2(Mask / No Mask)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(44944, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.layer3(out)
        return out


In [5]:
net = CNN()

In [10]:
# 확인
test_input = torch.Tensor(3,3,224,224) # batchsize, RGB channel, W, H
test_out = net(test_input)

## 손실함수와 Optimizer 정의
손실함수는 Cross-Entropy loss를 사용,
Optimizer는 SGD를 사용하였다.

In [7]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()


## 신경망 학습
7번 epoch으로 학습을 시킨다.
학습이 다되면, net.prm파일로 모델을 저장한다.


In [8]:
total_batch = len(train_set)


epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(train_set):
        imgs, labels = data
        imgs = imgs
        labels = labels
        optimizer.zero_grad() #변화도를 0으로 만든다.
        out = net(imgs)  
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')
params = net.state_dict()
torch.save(params, "net.prm", pickle_protocol=4)

[Epoch:1] cost = 0.5129538774490356
[Epoch:2] cost = 0.1684643030166626
[Epoch:3] cost = 0.05146336555480957
[Epoch:4] cost = 0.0310315303504467
[Epoch:5] cost = 0.022869283333420753
[Epoch:6] cost = 0.036764949560165405
[Epoch:7] cost = 0.013002562336623669
Learning Finished!


## 모델 로드
혹시 모델이 파일로 있다면 불러올수있다

In [17]:
params = torch.load("net.prm", map_location = "cpu")
net.load_state_dict(params)

<All keys matched successfully>

## 모델 평가 Validation
정확도를 확인하여 MASK 와 NO_Mask를 얼마나 잘 구분할수있는지 평가한다

In [9]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs
        label = label
        
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 0.9885714054107666


# 얼굴인식 OpenCV
cv2에 있는 라이브러리중 cascadeClassfier와 얼굴인식 haarcascade를 통해서 얼굴을 인식한다.


In [11]:
from keras.models import load_model
import cv2
import numpy as np

Using TensorFlow backend.


In [12]:
params = torch.load("net.prm", map_location = "cpu")
net.load_state_dict(params) 
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'with_mask',1:'without_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}


In [13]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=img[y:y+w,x:x+w]
        
        resized=cv2.resize(face_img,(224,224))
        
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,3,224,224))
        reshaped = reshaped.astype(np.float32)
        
        reshaped = torch.from_numpy(reshaped)
        
        result = net(reshaped)
        
        label = torch.argmax(result, 1)[0]
        label = label.item()
        #label=np.argmax(result, axis=1)[0]
       
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

KeyboardInterrupt: 

# Reference

https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html


https://towardsdatascience.com/real-time-face-mask-detector-with-tensorflow-keras-and-opencv-38b552660b64